In [3]:
import numpy as np

In [6]:
# UDP blocking probes from 2015 measurements
udp_blocking_2015 = \
np.array([   637.,    689.,    720.,   1004.,   1121.,   1270.,   2577.,
         2601.,   2844.,   2975.,   3021.,   3174.,   3730.,   4298.,
         4554.,   4720.,   4776.,   4833.,   4886.,  10212.,  10977.,
        11055.,  12289.,  13061.,  13240.,  13442.,  13712.,  13928.,
        14115.,  14390.,  14398.,  14419.,  14454.,  14593.,  14797.,
        14889.,  14998.,  15261.,  15342.,  15743.,  15782.,  15942.,
        15946.,  15955.,  16053.,  16414.,  16845.,  17009.,  17455.,
        17855.,  17938.,  18044.,  18182.,  18292.,  18354.,  18387.,
        18550.,  18648.,  19002.,  19201.,  19583.,  19656.,  19774.,
        19898.,  19953.,  19998.,  20105.,  20404.,  20423.,  20427.,
        20456.,  20479.,  20796.,  20861.,  21170.,  21903.,  22361.,
        22471.,  22667.,  22694.,  22948.,  23236.])

In [7]:
# UDP blocking probes from Emile's MTU measurements
udp_blocking_mar16 = \
np.array([    58.,     79.,    280.,    299.,    506.,    584.,    675.,
          679.,    817.,    843.,    861.,    892.,    980.,    996.,
         1004.,   1396.,   2087.,   2093.,   2162.,   2175.,   2235.,
         2237.,   2282.,   2333.,   2577.,   2690.,   2751.,   2792.,
         2844.,   2903.,   2906.,   2975.,   3004.,   3013.,   3083.,
         3156.,   3174.,   3216.,   3229.,   3280.,   3293.,   3303.,
         3370.,   3379.,   3457.,   3620.,   3639.,   3791.,   3828.,
         3831.,   3879.,   3906.,   4066.,   4072.,   4127.,   4151.,
         4203.,   4214.,   4270.,   4498.,   4554.,   4699.,   4776.,
         4814.,   4833.,   4875.,   4903.,   4965.,  10058.,  10212.,
        10250.,  10639.,  10915.,  11020.,  11056.,  11119.,  11232.,
        11359.,  11383.,  11410.,  11483.,  11530.,  11619.,  11802.,
        11832.,  11916.,  11978.,  11995.,  12050.,  12053.,  12402.,
        12778.,  12835.,  12880.,  12888.,  12974.,  12983.,  13051.,
        13061.,  13214.,  13219.,  13381.,  13442.,  13542.,  13558.,
        13633.,  13635.,  13742.,  13882.,  13928.,  13978.,  14273.,
        14376.,  14419.,  14450.,  14454.,  14635.,  14695.,  14847.,
        14877.,  14922.,  15037.,  15049.,  15063.,  15074.,  15252.,
        15261.,  15554.,  15625.,  15646.,  15743.,  15762.,  15817.,
        15862.,  15903.,  15942.,  15946.,  16006.,  16090.,  16106.,
        16216.,  16544.,  16548.,  16669.,  16689.,  16723.,  16759.,
        16821.,  16856.,  16958.,  17138.,  17176.,  17244.,  17428.,
        17505.,  17572.,  17807.,  17855.,  17895.,  17906.,  17917.,
        18000.,  18110.,  18148.,  18165.,  18243.,  18263.,  18273.,
        18296.,  18300.,  18344.,  18387.,  18627.,  18681.,  18695.,
        18697.,  18723.,  18904.,  18907.,  18984.,  19002.,  19055.,
        19201.,  19241.,  19298.,  19424.,  19434.,  19473.,  19505.,
        19569.,  19662.,  19693.,  19807.,  19813.,  19827.,  19898.,
        19953.,  19978.,  20025.,  20138.,  20342.,  20359.,  20400.,
        20404.,  20423.,  20430.,  20456.,  20461.,  20467.,  20476.,
        20488.,  20599.,  20630.,  20636.,  20757.,  20775.,  20777.,
        20843.,  20980.,  21074.,  21093.,  21165.,  21258.,  21320.,
        21351.,  21403.,  21424.,  21580.,  21704.,  21713.,  21726.,
        21764.,  21815.,  21882.,  21985.,  22112.,  22174.,  22510.,
        22568.,  22667.,  22813.,  22834.,  22880.,  22955.,  22991.,
        23012.,  23111.,  23180.,  23207.,  23267.,  23271.,  23272.,
        23281.,  23355.,  23403.,  23419.,  23467.,  23473.,  23749.,
        23857.,  23969.,  23986.,  23987.,  24170.,  24209.,  24268.,
        24310.,  24437.,  24503.,  24522.,  24748.,  24774.,  24826.,
        24891.,  25002.,  25089.,  25100.,  25160.,  25334.,  25377.,
        25595.,  25623.,  25960.,  26313.,  26389.,  26620.,  26774.,
        26808.,  26832.,  26908.,  26960.,  26993.,  27097.,  27320.,
        27325.,  27401.])